# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-05-02 16:33:08] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=37718, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=909492962, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_

[2025-05-02 16:33:23] Attention backend not set. Use fa3 backend by default.
[2025-05-02 16:33:23] Init torch distributed begin.


[2025-05-02 16:33:23] Init torch distributed ends. mem usage=0.00 GB
[2025-05-02 16:33:23] Load weight begin. avail mem=37.16 GB


[2025-05-02 16:33:23] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-02 16:33:24] Using model weights format ['*.safetensors']


...

[2025-05-02 16:33:53] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-02 16:33:54] INFO:     Started server process [2672826]
[2025-05-02 16:33:54] INFO:     Waiting for application startup.
[2025-05-02 16:33:54] INFO:     Application startup complete.
[2025-05-02 16:33:54] INFO:     Uvicorn running on http://0.0.0.0:37718 (Press CTRL+C to quit)


[2025-05-02 16:33:54] INFO:     127.0.0.1:44376 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-02 16:33:55] INFO:     127.0.0.1:44392 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-02 16:33:55] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-02 16:33:59] INFO:     127.0.0.1:44398 - "POST /generate HTTP/1.1" 200 OK
[2025-05-02 16:33:59] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Please generate the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-02 16:34:00] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-02 16:34:01] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 4.98, #queue-req: 0


[2025-05-02 16:34:02] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 57.63, #queue-req: 0


[2025-05-02 16:34:03] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 56.39, #queue-req: 0


[2025-05-02 16:34:03] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 56.33, #queue-req: 0


...

[2025-05-02 16:34:34] Decode batch. #running-req: 1, #token: 1932, token usage: 0.09, gen throughput (token/s): 58.78, #queue-req: 0


[2025-05-02 16:34:34] Decode batch. #running-req: 1, #token: 1972, token usage: 0.10, gen throughput (token/s): 58.96, #queue-req: 0


[2025-05-02 16:34:35] Decode batch. #running-req: 1, #token: 2012, token usage: 0.10, gen throughput (token/s): 59.15, #queue-req: 0


[2025-05-02 16:34:36] Decode batch. #running-req: 1, #token: 2052, token usage: 0.10, gen throughput (token/s): 59.22, #queue-req: 0


[2025-05-02 16:34:36] INFO:     127.0.0.1:44406 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Give me the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-02 16:34:36] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-02 16:34:37] Decode batch. #running-req: 1, #token: 44, token usage: 0.00, gen throughput (token/s): 48.18, #queue-req: 0


[2025-05-02 16:34:37] Decode batch. #running-req: 1, #token: 84, token usage: 0.00, gen throughput (token/s): 58.74, #queue-req: 0


[2025-05-02 16:34:38] Decode batch. #running-req: 1, #token: 124, token usage: 0.01, gen throughput (token/s): 58.95, #queue-req: 0


[2025-05-02 16:34:39] Decode batch. #running-req: 1, #token: 164, token usage: 0.01, gen throughput (token/s): 59.81, #queue-req: 0


...

[2025-05-02 16:35:08] Decode batch. #running-req: 1, #token: 1924, token usage: 0.09, gen throughput (token/s): 59.38, #queue-req: 0


[2025-05-02 16:35:09] Decode batch. #running-req: 1, #token: 1964, token usage: 0.10, gen throughput (token/s): 59.46, #queue-req: 0


[2025-05-02 16:35:10] Decode batch. #running-req: 1, #token: 2004, token usage: 0.10, gen throughput (token/s): 59.41, #queue-req: 0


[2025-05-02 16:35:11] Decode batch. #running-req: 1, #token: 2044, token usage: 0.10, gen throughput (token/s): 59.14, #queue-req: 0


[2025-05-02 16:35:11] INFO:     127.0.0.1:44406 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "user",
            "content": "Give me the information of the capital of France.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-02 16:35:11] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-02 16:35:11] Decode batch. #running-req: 1, #token: 39, token usage: 0.00, gen throughput (token/s): 53.81, #queue-req: 0


[2025-05-02 16:35:12] Decode batch. #running-req: 1, #token: 79, token usage: 0.00, gen throughput (token/s): 60.18, #queue-req: 0


[2025-05-02 16:35:13] Decode batch. #running-req: 1, #token: 119, token usage: 0.01, gen throughput (token/s): 59.14, #queue-req: 0


[2025-05-02 16:35:13] INFO:     127.0.0.1:44406 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-02 16:35:13] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-02 16:35:13] Decode batch. #running-req: 1, #token: 33, token usage: 0.00, gen throughput (token/s): 53.31, #queue-req: 0


[2025-05-02 16:35:14] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, gen throughput (token/s): 58.95, #queue-req: 0


[2025-05-02 16:35:15] Decode batch. #running-req: 1, #token: 113, token usage: 0.01, gen throughput (token/s): 60.07, #queue-req: 0


[2025-05-02 16:35:15] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, gen throughput (token/s): 59.27, #queue-req: 0


...

[2025-05-02 16:35:17] Decode batch. #running-req: 1, #token: 233, token usage: 0.01, gen throughput (token/s): 59.77, #queue-req: 0


[2025-05-02 16:35:17] Decode batch. #running-req: 1, #token: 273, token usage: 0.01, gen throughput (token/s): 57.43, #queue-req: 0


[2025-05-02 16:35:18] Decode batch. #running-req: 1, #token: 313, token usage: 0.02, gen throughput (token/s): 59.77, #queue-req: 0


[2025-05-02 16:35:19] Decode batch. #running-req: 1, #token: 353, token usage: 0.02, gen throughput (token/s): 59.88, #queue-req: 0


[2025-05-02 16:35:19] INFO:     127.0.0.1:44406 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "user",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-02 16:35:19] Prefill batch. #new-seq: 1, #new-token: 471, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-02 16:35:20] Decode batch. #running-req: 1, #token: 495, token usage: 0.02, gen throughput (token/s): 38.16, #queue-req: 0


[2025-05-02 16:35:21] Decode batch. #running-req: 1, #token: 535, token usage: 0.03, gen throughput (token/s): 54.39, #queue-req: 0


[2025-05-02 16:35:21] Decode batch. #running-req: 1, #token: 575, token usage: 0.03, gen throughput (token/s): 57.97, #queue-req: 0


[2025-05-02 16:35:22] Decode batch. #running-req: 1, #token: 615, token usage: 0.03, gen throughput (token/s): 58.38, #queue-req: 0


...

[2025-05-02 16:35:23] Decode batch. #running-req: 1, #token: 695, token usage: 0.03, gen throughput (token/s): 56.18, #queue-req: 0


[2025-05-02 16:35:24] Decode batch. #running-req: 1, #token: 735, token usage: 0.04, gen throughput (token/s): 55.73, #queue-req: 0


[2025-05-02 16:35:25] Decode batch. #running-req: 1, #token: 775, token usage: 0.04, gen throughput (token/s): 56.58, #queue-req: 0


[2025-05-02 16:35:25] Decode batch. #running-req: 1, #token: 815, token usage: 0.04, gen throughput (token/s): 57.06, #queue-req: 0


[2025-05-02 16:35:26] INFO:     127.0.0.1:44406 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "user",
        "content": "Here is the information of the capital of France in the JSON format.\n",
    }
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-05-02 16:35:26] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-02 16:35:27] Decode batch. #running-req: 1, #token: 47, token usage: 0.00, gen throughput (token/s): 30.78, #queue-req: 0


[2025-05-02 16:35:27] Decode batch. #running-req: 1, #token: 87, token usage: 0.00, gen throughput (token/s): 60.25, #queue-req: 0


[2025-05-02 16:35:28] Decode batch. #running-req: 1, #token: 127, token usage: 0.01, gen throughput (token/s): 60.11, #queue-req: 0


[2025-05-02 16:35:29] Decode batch. #running-req: 1, #token: 167, token usage: 0.01, gen throughput (token/s): 60.22, #queue-req: 0


...

[2025-05-02 16:35:59] Decode batch. #running-req: 1, #token: 1927, token usage: 0.09, gen throughput (token/s): 57.63, #queue-req: 0


[2025-05-02 16:36:00] Decode batch. #running-req: 1, #token: 1967, token usage: 0.10, gen throughput (token/s): 58.63, #queue-req: 0


[2025-05-02 16:36:00] Decode batch. #running-req: 1, #token: 2007, token usage: 0.10, gen throughput (token/s): 58.05, #queue-req: 0


[2025-05-02 16:36:01] Decode batch. #running-req: 1, #token: 2047, token usage: 0.10, gen throughput (token/s): 57.55, #queue-req: 0


[2025-05-02 16:36:02] INFO:     127.0.0.1:36160 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so I need to provide the information about the capital of France in JSON format. Hmm, I\'m not entirely sure about all the details, but I\'ll try to think it through.\n\nFirst, I know that the capital of France is Paris. That\'s pretty much a given, right? But I should double-check that. Maybe I can recall any other capitals I know. London is the capital of the UK, Rome is Italy, and maybe Tokyo is Japan\'s. Yeah, Paris seems correct for France.\n\nNow, moving on to the population. I think Paris is a very large city, but I\'m not sure of the exact number. I remember it\'s over 3 million, but I\'m not certain. Maybe around 3.5 million? I should probably look that up, but since I can\'t right now, I\'ll go with 3,500,000 as an estimate.\n\nNext, the area. Paris is a big city, but I think it\'s not as large as Tokyo or London. Maybe around 10 square kilometers? I\'m not sure, but that seems 

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=True)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-05-02 16:36:02] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-02 16:36:02] Decode batch. #running-req: 1, #token: 24, token usage: 0.00, gen throughput (token/s): 56.07, #queue-req: 0


[2025-05-02 16:36:03] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, gen throughput (token/s): 59.39, #queue-req: 0


[2025-05-02 16:36:03] Decode batch. #running-req: 1, #token: 104, token usage: 0.01, gen throughput (token/s): 58.69, #queue-req: 0


[2025-05-02 16:36:04] Decode batch. #running-req: 1, #token: 144, token usage: 0.01, gen throughput (token/s): 52.38, #queue-req: 0


...

[2025-05-02 16:36:12] Decode batch. #running-req: 1, #token: 624, token usage: 0.03, gen throughput (token/s): 57.40, #queue-req: 0


[2025-05-02 16:36:13] Decode batch. #running-req: 1, #token: 664, token usage: 0.03, gen throughput (token/s): 60.07, #queue-req: 0


[2025-05-02 16:36:14] Decode batch. #running-req: 1, #token: 704, token usage: 0.03, gen throughput (token/s): 59.97, #queue-req: 0


[2025-05-02 16:36:14] Decode batch. #running-req: 1, #token: 744, token usage: 0.04, gen throughput (token/s): 59.77, #queue-req: 0


[2025-05-02 16:36:15] Decode batch. #running-req: 1, #token: 784, token usage: 0.04, gen throughput (token/s): 58.19, #queue-req: 0
[2025-05-02 16:36:15] INFO:     127.0.0.1:53858 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-05-02 16:36:15] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-02 16:36:15] Prefill batch. #new-seq: 2, #new-token: 2, #cached-token: 20, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-02 16:36:15] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 10, token usage: 0.00, #running-req: 2, #queue-req: 0


[2025-05-02 16:36:16] Decode batch. #running-req: 3, #token: 95, token usage: 0.00, gen throughput (token/s): 92.61, #queue-req: 0


[2025-05-02 16:36:17] Decode batch. #running-req: 3, #token: 215, token usage: 0.01, gen throughput (token/s): 166.59, #queue-req: 0


[2025-05-02 16:36:17] Decode batch. #running-req: 3, #token: 335, token usage: 0.02, gen throughput (token/s): 167.23, #queue-req: 0


[2025-05-02 16:36:18] Decode batch. #running-req: 3, #token: 455, token usage: 0.02, gen throughput (token/s): 165.37, #queue-req: 0


[2025-05-02 16:36:19] Decode batch. #running-req: 3, #token: 575, token usage: 0.03, gen throughput (token/s): 166.28, #queue-req: 0


[2025-05-02 16:36:19] Decode batch. #running-req: 1, #token: 239, token usage: 0.01, gen throughput (token/s): 96.99, #queue-req: 0


[2025-05-02 16:36:20] INFO:     127.0.0.1:50186 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThat's all the information I have.\n\nOkay, so I need to figure out the capital of France. I know that Paris is the capital, but I'm not entirely sure. Let me think about why I think that. I've heard it mentioned a lot, especially in movies and TV shows. People often go there for business or tourism. Also, I remember learning in school that Paris is a major city in France, known for landmarks like the Eiffel Tower and the Louvre Museum. Those places are famous worldwide, which makes me think that Paris is indeed the capital. Maybe I can cross-check this with some other sources or my notes. Wait, I don't have any other information right now, but based on what I know, Paris is the capital of France. I don't recall any other major city in France being referred to as the capital. So, I'm pretty confident that Paris is correct.\n</think>Paris is the capital of F

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-05-02 16:36:20] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-02 16:36:20] Decode batch. #running-req: 1, #token: 17, token usage: 0.00, gen throughput (token/s): 54.93, #queue-req: 0


[2025-05-02 16:36:21] Decode batch. #running-req: 1, #token: 57, token usage: 0.00, gen throughput (token/s): 57.12, #queue-req: 0


[2025-05-02 16:36:22] Decode batch. #running-req: 1, #token: 97, token usage: 0.00, gen throughput (token/s): 58.92, #queue-req: 0


[2025-05-02 16:36:22] Decode batch. #running-req: 1, #token: 137, token usage: 0.01, gen throughput (token/s): 58.77, #queue-req: 0


...

[2025-05-02 16:36:53] Decode batch. #running-req: 1, #token: 1897, token usage: 0.09, gen throughput (token/s): 58.02, #queue-req: 0


[2025-05-02 16:36:53] Decode batch. #running-req: 1, #token: 1937, token usage: 0.09, gen throughput (token/s): 58.94, #queue-req: 0


[2025-05-02 16:36:54] Decode batch. #running-req: 1, #token: 1977, token usage: 0.10, gen throughput (token/s): 59.18, #queue-req: 0


[2025-05-02 16:36:55] Decode batch. #running-req: 1, #token: 2017, token usage: 0.10, gen throughput (token/s): 59.71, #queue-req: 0


[2025-05-02 16:36:55] INFO:     127.0.0.1:50194 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-05-02 16:36:55] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 19, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-02 16:36:55] Decode batch. #running-req: 1, #token: 23, token usage: 0.00, gen throughput (token/s): 56.92, #queue-req: 0


[2025-05-02 16:36:56] Decode batch. #running-req: 1, #token: 63, token usage: 0.00, gen throughput (token/s): 60.21, #queue-req: 0


[2025-05-02 16:36:57] Decode batch. #running-req: 1, #token: 103, token usage: 0.01, gen throughput (token/s): 60.31, #queue-req: 0


[2025-05-02 16:36:57] Decode batch. #running-req: 1, #token: 143, token usage: 0.01, gen throughput (token/s): 60.01, #queue-req: 0


[2025-05-02 16:36:58] Decode batch. #running-req: 1, #token: 183, token usage: 0.01, gen throughput (token/s): 60.13, #queue-req: 0


...

[2025-05-02 16:37:10] Decode batch. #running-req: 1, #token: 863, token usage: 0.04, gen throughput (token/s): 56.93, #queue-req: 0


[2025-05-02 16:37:10] Decode batch. #running-req: 1, #token: 903, token usage: 0.04, gen throughput (token/s): 57.27, #queue-req: 0


[2025-05-02 16:37:11] Decode batch. #running-req: 1, #token: 943, token usage: 0.05, gen throughput (token/s): 57.83, #queue-req: 0


[2025-05-02 16:37:12] Decode batch. #running-req: 1, #token: 983, token usage: 0.05, gen throughput (token/s): 57.62, #queue-req: 0


[2025-05-02 16:37:13] Decode batch. #running-req: 1, #token: 1023, token usage: 0.05, gen throughput (token/s): 55.05, #queue-req: 0
[2025-05-02 16:37:13] INFO:     127.0.0.1:52770 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:03<00:03,  3.14s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:07<00:00,  3.80s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:07<00:00,  3.70s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure, here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Tiananmen Square",
    "Forbidden City",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
I know that the capital of France is Paris, but I need more details. Can you provide me with some information about Paris?

Of course, Paris is the capital of France. It's a very famous city. Let me think about what else I know about Paris. It's located in the northern part of the country. It's known for its rich history, beautiful landmarks, and vibrant culture. I remember the Eiffel Tower is there, that's a big attraction. Also, the Louvre Museum is another famous place. There's also Notre-Dame Cathedral. Paris is known for its art, fashion, and cuisine.
Prompt: Give me the information of the capital of Germany.
Generated text:  the capital of Germany is Berlin. So, if I have a question about the capital of Germany, I should answer it with information about Berlin. I need to use the information given to answer the question.
Okay, so I have to figure out the capital of Germany. Hmm, I think I remember learning 

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its population, location, economic status, cultural significance, major landmarks, and transportation network. Let's see if we can add something about the global economy.

First, I need to find the most recent population estimates for London. I believe the population has been increasing steadily, so maybe around 9 million? I should check a reliable source to confirm.

Next, location: London is located in England, United Kingdom, at approximately 51.5 degrees north latitude and 0 degrees longitude. It's on the River Thames, which has been crucial for its development. The proximity to water has facilitated trade and transportation historically.

Economically
Prompt: Please provide information about Paris as a major global city:
Generated text:  its location, economic status, cultural significance, and a famous attraction.

8-10 sentences, in English, no markdown.
Okay, so I need to find out informati

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜User｜>Here is the information of the capital of France in the JSON format.
<｜Assistant｜><think>

Generated text: Alright, so the user is asking for information about the capital of France. I know that Paris is the capital, so that's a given. But they want it in JSON format, so I should structure it properly. I'll include the country, city name, population, and maybe the region or area it's located in. I should make sure the JSON syntax is correct with proper commas and brackets. I don't want any typos or missing fields. Once I have all that, I can format it neatly in the response.
</think>

Here is the information about the capital of France in JSON format:

```json
{
  "country": "France",
  "city": "Paris",
  "population": "9,221,400",
  "region": "Ile de France"
}
```


In [19]:
llm.shutdown()